# MetroPT-3 Dataset

El **MetroPT-3 Dataset** es un conjunto de datos multivariado y temporal diseñado para tareas de mantenimiento predictivo, detección de anomalías y estimación de vida útil remanente (RUL) en compresores de unidades de producción de aire (APU) de trenes. Los datos se registraron entre febrero y agosto de 2020 a una frecuencia de 1Hz, proporcionando 15 características extraídas de sensores analógicos y digitales. Contiene información sin etiquetar, pero se acompañan reportes de fallas para evaluar modelos.

---

## Descripción de las columnas (atributos):

- **TP2 (bar):** Presión en el compresor.
- **TP3 (bar):** Presión generada en el panel neumático.
- **H1 (bar):** Presión causada por la caída al descargar el filtro separador ciclónico.
- **DV Pressure (bar):** Presión de descarga en secadores de aire (valor 0 indica operación bajo carga).
- **Reservoirs (bar):** Presión en los depósitos, debe aproximarse a TP3.
- **Motor Current (A):** Corriente de un motor trifásico:
  - `0A`: Apagado.
  - `4A`: Sin carga.
  - `7A`: Bajo carga.
  - `9A`: Encendido.
- **Oil Temperature (ºC):** Temperatura del aceite en el compresor.
- **COMP:** Señal eléctrica de la válvula de admisión de aire; indica estado apagado o sin carga.
- **DV Electric:** Señal eléctrica que controla la válvula de salida del compresor; activa bajo carga.
- **TOWERS:** Señal que identifica la torre activa para secado o drenaje de humedad:
  - **Inactiva:** Torre 1.
  - **Activa:** Torre 2.
- **MPG:** Señal para iniciar el compresor bajo carga si la presión cae por debajo de 8.2 bar.
- **LPS:** Señal que activa cuando la presión baja de 7 bar.
- **Pressure Switch:** Señal que detecta descargas en torres de secado de aire.
- **Oil Level:** Señal que detecta niveles de aceite bajos.
- **Caudal Impulse:** Pulso generado por el flujo de aire hacia los depósitos.

In [1]:
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt

# Lectura y descripción de datos
Leemos los datos y sacamos estadísticos básicos, así como porcentaje de nulos y alguna limpieza básica inicial.

In [2]:
# ----- Read data ----- #
csv_path = "MetroPT3(AirCompressor).csv"
data = pl.read_csv(csv_path)

# ----- Summarizing data ----- #
print(data.shape)
print(data.columns)
print(data.describe())
print(data.null_count())
for col, dtype in zip(data.columns, data.dtypes):
    print(f"Column: {col}, Type: {dtype}")

# ----- Cleaning data ----- #
# Columns that are not features
data = data.drop(pl.col(""), pl.col("timestamp"))
print(len(data.columns))
print(data.describe())

(1516948, 17)
['', 'timestamp', 'TP2', 'TP3', 'H1', 'DV_pressure', 'Reservoirs', 'Oil_temperature', 'Motor_current', 'COMP', 'DV_eletric', 'Towers', 'MPG', 'LPS', 'Pressure_switch', 'Oil_level', 'Caudal_impulses']
shape: (9, 18)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ statistic ┆           ┆ timestamp ┆ TP2       ┆ … ┆ LPS       ┆ Pressure_ ┆ Oil_level ┆ Caudal_i │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ switch    ┆ ---       ┆ mpulses  │
│ str       ┆ f64       ┆ str       ┆ f64       ┆   ┆ f64       ┆ ---       ┆ f64       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆           ┆ f64       ┆           ┆ f64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ count     ┆ 1.516948e ┆ 1516948   ┆ 1.516948e ┆ … ┆ 1.516948e ┆ 1.516948e ┆ 1.516948e ┆ 1.516948 │
│           ┆ 6         ┆           ┆ 6         ┆   ┆ 6         